In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=5c7d8cbd427b0a8353b3b6c8fb45acf7ba185fc795bfa2c2ac3d7fc2667230ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-ykfsusyj/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator, BucketIterator

In [ ]:
import torch
import torchtext
# from torchtext.data import Field, LabelField # For torch<=0.8.0, the importing of functions should be `from torchtext.data`
# from torchtext.data import TabularDataset
# from torchtext.data import Iterator, BucketIterator
import spacy
import en_core_web_lg
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

cpu


In [140]:
spacy_en = en_core_web_lg.load()

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

  

In [141]:
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)

train, val = TabularDataset.splits(
               path="./drive/My Drive/CNN/", # the root directory where the data lies
               train='train_sentiment.tsv', validation="val_sentiment.tsv", # file names
               format='tsv',
               skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=[('tweet', TEXT), ('label', LABEL)])

In [ ]:
test = TabularDataset.splits(
               path="/content/drive/MyDrive/CNN/", # the root directory where the data lies
               test="test_masks.tsv", # file names
               format='tsv',
               skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=[(None, None), ('tweet', TEXT), ('label', LABEL)])

In [ ]:
test = test[0]

In [ ]:
# import torchtext.vocab as vocab
# custom_embeddings = vocab.Vectors(name = root_path + 'glove.twitter.27B.200d.txt',
#                                   cache = 'custom_embeddings',
#                                   unk_init = torch.Tensor.normal_)
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "./drive/My Drive/CNN/emb.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
glove.twitter.27B.100d.txt                     2015-12-22 16:04:54   1021671926
glove.twitter.27B.200d.txt                     2015-12-22 16:04:54   2057595650
glove.twitter.27B.25d.txt                      2015-12-22 16:04:54    257699930
glove.twitter.27B.50d.txt                      2015-12-22 16:04:54    510889212
Extracting all the files now...
Done!


In [ ]:

import torchtext.vocab as vocab
loaded_vectors = torchtext.vocab.Vectors('/content/drive/MyDrive/CNN/glove.twitter.27B.200d.txt')
TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
LABEL.build_vocab(train)
print("Vocabulary size of TEXT:",len(TEXT.vocab.stoi))
print("Vocabulary size of LABEL:",len(LABEL.vocab.stoi))

100%|█████████▉| 1193516/1193517 [01:43<00:00, 11511.41it/s]


Vocabulary size of TEXT: 50258
Vocabulary size of LABEL: 3


In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), 
    batch_size = 32,
    sort_key=lambda x: len(x.tweet), 
    sort=True,
    sort_within_batch=True,
    device = device)

In [ ]:
class CNN_BiLSTM(nn.Module):

    def __init__(self, args):
        super(CNN_BiLSTM, self).__init__()
        self.args = args
        self.hidden_dim = args.lstm_hidden_dim
        self.num_layers = args.lstm_num_layers
        V = args.embed_num
        D = args.embed_dim
        C = args.class_num
        self.C = C
        Ci = 1
        Co = args.kernel_num
        Ks = args.kernel_sizes
        self.embed = nn.Embedding(V, D, padding_idx=args.paddingId)
        # pretrained  embedding
        if args.word_Embedding:
            self.embed.weight.data.copy_(args.pretrained_weight)

        # CNN
        self.convs1 = [nn.Conv2d(Ci, Co, (K, D), padding=(K//2, 0), stride=1) for K in Ks]
        print(self.convs1)
        # for cnn cuda
        # if self.args.cuda is True:
        #     for conv in self.convs1:
        #         conv = conv.cuda()

        # BiLSTM
        self.bilstm = nn.LSTM(D, self.hidden_dim, num_layers=self.num_layers, dropout=args.dropout, bidirectional=True, bias=True)

        # linear
        L = len(Ks) * Co + self.hidden_dim * 2
        self.hidden2label1 = nn.Linear(L, L // 2)
        self.hidden2label2 = nn.Linear(L // 2, C)

        # dropout
        self.dropout = nn.Dropout(args.dropout)

    def forward(self, x):
        embed = self.embed(x)

        # CNN
        cnn_x = embed
        cnn_x = torch.transpose(cnn_x, 0, 1)
        cnn_x = cnn_x.unsqueeze(1)
        cnn_x = [F.relu(conv(cnn_x)).squeeze(3) for conv in self.convs1]  # [(N,Co,W), ...]*len(Ks)
        cnn_x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in cnn_x]  # [(N,Co), ...]*len(Ks)
        cnn_x = torch.cat(cnn_x, 1)
        cnn_x = self.dropout(cnn_x)

        # BiLSTM
        bilstm_x = embed.view(len(x), embed.size(1), -1)
        bilstm_out, _ = self.bilstm(bilstm_x)
        bilstm_out = torch.transpose(bilstm_out, 0, 1)
        bilstm_out = torch.transpose(bilstm_out, 1, 2)
        bilstm_out = F.max_pool1d(bilstm_out, bilstm_out.size(2)).squeeze(2)
        # bilstm_out = F.tanh(bilstm_out)

        # CNN and BiLSTM CAT
        cnn_x = torch.transpose(cnn_x, 0, 1)
        bilstm_out = torch.transpose(bilstm_out, 0, 1)
        cnn_bilstm_out = torch.cat((cnn_x, bilstm_out), 0)
        cnn_bilstm_out = torch.transpose(cnn_bilstm_out, 0, 1)

        # linear
        cnn_bilstm_out = self.hidden2label1(F.tanh(cnn_bilstm_out))
        cnn_bilstm_out = self.hidden2label2(F.tanh(cnn_bilstm_out))

        # output
        logit = cnn_bilstm_out
        return logit

In [171]:
# vocabulary_size = len(TEXT.vocab.stoi) 
# embedding_dim = 200 

# kernels_num = 32 
# output_size = 3
       
# PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
# num_epoch = 20 

class CNNArgs:
  def __init__(self, hidden_dim, num_layers, vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout, padIdx) -> None:
      self.lstm_hidden_dim = hidden_dim
      self.lstm_num_layers = num_layers
      self.embed_num = vocabulary_size 
      self.embed_dim = embedding_dim
      self.class_num = output_size
      self.kernel_num = kernels_num
      self.kernel_sizes = kernel_sizes
      self.dropout = dropout
      self.paddingId = padIdx
      self.word_Embedding = None
      self.cuda = True

def random_search(num_iter):
    results = []
    for i in range(num_iter):
      TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
      LABEL = Field(sequential=False, unk_token = None) 
      train, val = TabularDataset.splits(
                      path="./drive/My Drive/CNN/",
                      train='train_optm_sentiment.tsv', validation="val_sentiment.tsv",
                      format='tsv',
                      skip_header=True,
                      fields=[('tweet', TEXT), ('label', LABEL)])
        
      loaded_vectors = torchtext.vocab.Vectors('/content/drive/MyDrive/CNN/glove.twitter.27B.200d.txt')
      TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
      TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
      LABEL.build_vocab(train)
      # print("Vocabulary size of TEXT:",len(TEXT.vocab.stoi))
      # print("Vocabulary size of LABEL:",len(LABEL.vocab.stoi))

      train_iter, val_iter = data.BucketIterator.splits(
            (train, val), 
            batch_size = 32,
            sort_key=lambda x: len(x.tweet), 
            sort=True,
            sort_within_batch=True,
            device = device)
      # print(len(train_iter))
      # print(len(val_iter))
      config = {
            "num_layers" : random.choice([1, 2, 3, 4]),
            "kernel_sizes": random.choice([[1,2,3],[2,3,4],[3,4,5],[4,5,6],[1,4,6],[1,3,6],[2,5,4]]),
            "hidden_dim": random.choice([300, 400, 500, 600, 700, 800]),
            "dropout" : random.choice([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]),
            "lr": random.choice([0.1, 0.01, 0.001, 0.0001, 0.00001])     
        }
        
      print("new config")
      print(config)

      args_for_cnn = CNNArgs(hidden_dim=config["hidden_dim"], num_layers=config["num_layers"], vocabulary_size=len(TEXT.vocab.stoi),
                               embedding_dim=200, output_size=3, kernels_num=32, kernel_sizes=config["kernel_sizes"], 
                               dropout=config["dropout"], padIdx=TEXT.vocab.stoi[TEXT.pad_token])
      model = CNN_BiLSTM(args_for_cnn).to(device)

      pretrained_embeddings = TEXT.vocab.vectors
      model.embed.weight.data.copy_(pretrained_embeddings)
      UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
      model.embed.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
      model.embed.weight.data[PAD_IDX] = torch.zeros(embedding_dim)
      optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
      criterion = nn.CrossEntropyLoss()
      model.to(device)

      def train(model, iterator, optimizer, criterion):
        model.train()
        epoch_loss = 0
        for i, batch in enumerate(iterator):
          batch_input, labels = batch.tweet, batch.label
          batch_input = batch_input.to(device)
          labels = labels.to(device)
          optimizer.zero_grad()
          outputs = model(batch_input)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          epoch_loss += loss.cpu().item()
        return epoch_loss / len(iterator)

      def evaluate(model, iterator, criterion):
        model.eval()
        epoch_loss = 0
        all_pred=[]
        all_label = []
        with torch.no_grad():
            for i, batch in enumerate(iterator):
                batch_input, labels = batch.tweet, batch.label
                batch_input = batch_input.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(batch_input)
                loss = criterion(outputs, labels)
                epoch_loss += loss.cpu().item()
                # identify the predicted class for each example in the batch
                probabilities, predicted = torch.max(outputs.cpu().data, 1)
                # put all the true labels and predictions to two lists
                all_pred.extend(predicted)
                all_label.extend(labels.cpu())
        accuracy = accuracy_score(all_label, all_pred)
        f1score = f1_score(all_label, all_pred, average='macro') 
        return epoch_loss / len(iterator), accuracy, f1score

      max_val = 0
      best_epoch = 0
      for epoch in range(num_epoch):
        # train the model for one pass over the data
        train_loss = train(model, train_iter, optimizer, criterion) 
        val_loss, val_acc, val_f1 = evaluate(model, val_iter, criterion)
        if val_acc > max_val:
          max_val = val_acc
          best_epoch = epoch+1
        print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, num_epoch, train_loss, val_loss, val_acc, val_f1))

      print(f'best epoch: {best_epoch}, max val: {max_val}, config: {config}')
      results.append((max_val,best_epoch,config))
    return results

In [ ]:
# {'num_layers': 1, 'kernel_sizes': [1, 2, 3], 'hidden_dim': 700, 'dropout': 0.5, 'lr': 0.001} - 0.65 0.63
r=random_search(30)
r

new config
{'num_layers': 3, 'kernel_sizes': [3, 4, 5], 'hidden_dim': 300, 'dropout': 0.7, 'lr': 0.1}
[Conv2d(1, 32, kernel_size=(3, 200), stride=(1, 1), padding=(1, 0)), Conv2d(1, 32, kernel_size=(4, 200), stride=(1, 1), padding=(2, 0)), Conv2d(1, 32, kernel_size=(5, 200), stride=(1, 1), padding=(2, 0))]


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



 Epoch [1/20], Train Loss: 4.3601, Validation Loss: 1.1334, Validation Accuracy: 0.4345, Validation F1: 0.2019


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



 Epoch [2/20], Train Loss: 3.1046, Validation Loss: 3.9016, Validation Accuracy: 0.4345, Validation F1: 0.2019


...

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



 Epoch [1/20], Train Loss: 16.1372, Validation Loss: 25.8740, Validation Accuracy: 0.4095, Validation F1: 0.1937


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



 Epoch [2/20], Train Loss: 11.2666, Validation Loss: 4.9331, Validation Accuracy: 0.1560, Validation F1: 0.0900


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
